In [ ]:
import os
import tasti
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

os.environ['TORCH_HOME'] = '/lfs/1/jtguibas/models'
os.environ['FVCORE_CACHE'] = '/lfs/1/jtguibas/models'

In [ ]:
video = tasti.Video('/lfs/1/jtguibas/data/2017-12-17')

In [ ]:
frame = video[0]

In [ ]:
plt.imshow(frame.permute(1, 2, 0))

In [ ]:
cfg_fp = 'Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml'
cfg = detectron2.config.get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(cfg_fp))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(cfg_fp)
model = build_model(cfg)
DetectionCheckpointer(model).load(model_zoo.get_checkpoint_url(cfg_fp))
model.eval()

In [ ]:
inputs = [{
    'image': frame,
    'height': 1080,
    'width': 1920,
}]
output = model(inputs)

In [ ]:
frame.unsqueeze(0).shape